In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import START, MessagesState, StateGraph

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")


def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


builder = StateGraph(MessagesState)
builder.add_node(call_model)
builder.add_edge(START, "call_model")

graph = builder.compile()

In [2]:
config = {"configurable": {"thread_id": "0"}}

for chunk in graph.stream(
    {"messages": [{"role": "user", "content": "What is the capital of Germany?"}]},
    config,
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

What is the capital of Germany?
================================== Ai Message ==================================

Berlin


In [3]:
for chunk in graph.stream(
    {"messages": [{"role": "user", "content": "What was my previous question?"}]},
    config,
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

What was my previous question?
================================== Ai Message ==================================

As an AI, I have no memory of past conversations.  Each interaction starts fresh.  To know your previous question, you'll need to check your chat history.


In [4]:
graph = builder.compile(checkpointer=InMemorySaver())

In [5]:
config = {"configurable": {"thread_id": "0"}}

for chunk in graph.stream(
    {"messages": [{"role": "user", "content": "What is the capital of Germany?"}]},
    config,
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

What is the capital of Germany?
================================== Ai Message ==================================

Berlin


In [6]:
for chunk in graph.stream(
    {"messages": [{"role": "user", "content": "What was my previous question?"}]},
    config,
    stream_mode="values",
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

What was my previous question?
================================== Ai Message ==================================

Your previous question was: "What is the capital of Germany?"
